In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp

objc[1775]: Class CaptureDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1767ea538) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x144630860). One of the two will be used. Which one is undefined.
objc[1775]: Class CVWindow is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1767ea588) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x12760ca68). One of the two will be used. Which one is undefined.
objc[1775]: Class CVView is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1767ea5b0) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_h

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results = model.process(image)
    image.flags.writeable=True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret, frame = cap.read()
    image, results = mediapipe_detection(frame, holistic)
    print(results)
    draw_landmarks(image,results)
    cv2.imshow('OpenCV Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks.landmark else np.zeros(33)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x,res.y,res.z,] for res in results.face_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(1404)
    return np.concatenate([pose,face,lh,rh])

In [6]:
result_test = extract_keypoints(results)

NameError: name 'results' is not defined

In [ ]:
np.save('0',result_test)

In [ ]:
np.load('0.npy')

In [ ]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['hello'])
no_sequences = 20
sequence_length = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  for action in actions:
    for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            draw_landmarks(image,results)
            if frame_num == 0:
                cv2.putText(image,'STARTING COLLECTION',(120,200), cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),1, cv2.LINE_AA)
                cv2.putText(image,'Collecting frames for {} Video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1, cv2.LINE_AA)
                cv2.waitKey(2000)
            else:
                cv2.putText(image,'Collecting frames for {} Video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1, cv2.LINE_AA)
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
            np.save(npy_path,keypoints)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()